# 10

This part considers finite difference approximations and numerical solutions of ODEs. Lets start with the initializations as usual.

In [ ]:
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

mpl.rcParams['figure.figsize'] = (5.0, 5.0)

%matplotlib inline

- - -

<div class="alert alert-info">
    
## Finite Difference Approximations
</div>

For this exercise, we consider the following function $f:\mathbb{R}^2 \rightarrow \mathbb{R}$ defined by

$$
f(x,y) = \sin(x+y) + (x-y)^2 -1.5x + 2.5y +1
$$

As usual we can plot the graph of this function, i.e. the surface over the $(x,y)$-plane whose height is given by $f(x,y)$, over the rectangle $[-2,2]^2$:

In [ ]:
def func(x, y):
    """evaluate f at (x,y)"""
    return np.sin(x+y) + (x-y)**2 - 1.5*x + 2.5*y+1

# define a grid 
X, Y = np.mgrid[-2:2:100j,-2:2:100j]

# make a surface plot of the graph of f
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection='3d')

ax.plot_surface(X, Y, func(X, Y), cmap="viridis"); 

<div class="alert alert-success">
    
**Task 1:** Implement a function `gradient` that computes the gradient of the function `func` at $x,y$ using symbolic differentiation rules.
</div>

In [ ]:
def gradient(x, y):
    """evaluate the gradient of f at (x,y)"""
    dx = np.cos(x+y) + 2*(x-y) - 3/2.0
    dy = np.cos(y+x) - 2*(x-y) + 5/2.0
    
    return np.array([dx, dy])

In practice, we are rarely in the fortunate situation that finding gradients analytically is feasible and efficient. In these cases, we rather approximate the derivative, e.g. using finite difference approximations.

<div class="alert alert-success">
    
**Task 2:** Write the functions `gradient_fd_forward` and `central_fd_central` that compute the gradient of the function `func` at $x,y$ using the forward and centered finite differences with stepsize $h$.
</div>

In [ ]:
# from sympy import *
# xf = symbols('x')
# yf = symbols('y')

def gradient_fd_forward(func, x, y, h=0.1):
    """compute the gradient of func at x,y with forward finite differences of stepsize h"""
    # fdx = ( func(x+(2*h), y) - 2*func(x+h, y) + func(x, y) ) / h**2
    # fdy = ( func(x, y+(2*h)) - 2*func(x, y+h) + func(x, y) ) / h**2

    # first order accuracy mit O(h)
    fdx = (func(x+h, y) - func(x, y)) / h
    fdy = (func(x, y+h) - func(x, y)) / h
    
    return np.array([fdx, fdy])
    
def central_fd_central(func, x, y, h=0.1):
    """compute the gradient of func at x,y with centered finite differences of stepsize h"""
    # second order accuracy mit O(h^2)
    # fdx = ( func(x+h, y) - 2*func(x, y) + func(x-h, y) ) / h**2
    # fdy = ( func(x, y+h) - 2*func(x, y) + func(x, y-h) ) / h**2

    fdx = (func(x+h, y) - func(x-h, y)) / h*2
    fdy = (func(x, y+h) - func(x, y-h)) / h*2
    
    return np.array([fdx, fdy])

<div class="alert alert-success">
    
**Task 3:** Compare the analytical gradient (`gradient`) and approximate gradients (`gradient_fd_forward`, `gradient_fd_central`) for three choices of $h \in \{ 0.01, 0.1, 0.5 \}$ using the quiver plots (vector plots).
</div>

In [ ]:
# define a grid
# x_pts, y_pts = np.mgrid[-5:5:200j,-5:5:200j] # we just used the same grid points as earlier -2:2:100j,-2:2:100j
x_pts, y_pts = np.mgrid[-2:2:100j,-2:2:100j]

# get the values from calculating the gradient
gradient_x, gradient_y = gradient(x_pts, y_pts)
# plot function ontop of the contours
# plt.contour(x_pts, y_pts, func(x_pts, y_pts))

# plot for analytical gradient
plt.figure(figsize=(7,7)).add_subplot(111).quiver(x_pts, y_pts, gradient_x, gradient_y, color='y', label='Analytical gradient')
plt.legend(loc=(1.01, 0.935))

# plot for approximate gradients for three choices of h
for h, color in zip([0.01, 0.1, 0.5, 1], ['r', 'g', 'b', 'm']):
    fwd_fdx, fwd_fdy = gradient_fd_forward(func, x_pts, y_pts, h)
    ctr_fdx, ctr_fdy = central_fd_central(func, x_pts, y_pts, h)
    plt.figure(figsize=(7,7)).add_subplot(111).quiver(x_pts, y_pts, fwd_fdx, fwd_fdy, color=color, alpha=0.5, label=f'Gradient_fd_forward, h={h}')
    plt.legend(loc=(1.01, 0.935))
    plt.figure(figsize=(7,7)).add_subplot(111).quiver(x_pts, y_pts, ctr_fdx, ctr_fdy, color=color, alpha=0.5, label=f'Gradient_fd_central, h={h}')
    plt.legend(loc=(1.01, 0.935))

# plt.legend(loc=(1.01, 0.61)) # set the legend on the far right
plt.show()

<div class="alert alert-info">
    
## Numerical Solution of ODEs: Lotka-Volterra Equations
</div>

In the lecture, we have seen several numerical methods to solve ODEs. In this exercise, we are going to try them for the Lotka-Volterra equations. These are a system of ODEs that stem from ecology. In the following, we briefly explain the Lotka-Volterra model. 

The classic Lotka-Volterra model describes the population dynamics involving two species. One species is the prey species (e.g., rabbits) and its population size is represented by the variable $x$. The other species is the predator species (e.g., foxes), whose population size will be represented by $y$. The Lotka-Volterra model is used to predict the population sizes of prey and predator over time.

There are three important effects considered in the Lotka-Volterra system. 

The first effect is reproduction of the prey species. It is assumed that reproduction is proportional to the population, which means that $\frac{d x}{dt}= \alpha x$. The variable $\alpha$ is a positive constant. 

The second effect is the starvation of the predator species. If there are not adequate amount of preys in the system, some of the predator species will die due to starvation. This is expressed mathematically as $\frac{d y}{dt}= - \gamma y$, where $\gamma$ is another positive constant. 

According to these two equations, the two species would not interact. Finally, the last effect is the predation, i.e., the consumption of the prey species by the predator species. Note that without predators, the prey population would grow exponentially in time. Therefore, in this model, the predation keeps the prey population in balance. Similarly, without any prey, the predator species would simply die out. The predation effect is modeled by an additional bilinear term for the time derivatives of both the prey and predator population: $\frac{d x}{dt}$ gets an additional summand $-\beta xy$ and $\frac{d y}{dt}$ gets an additional summand $\delta xy$, where $\beta$ and $\delta$ are also positive constants. 

Taking these three effects into account, the time rate of change in each population can be represented by the following non-linear system of ODEs:
\begin{equation*}
\frac{d x}{dt}  =  \alpha x -\beta xy \quad \mbox{and} \quad \frac{d y}{dt} = - \gamma y + \delta xy.
\end{equation*}

<div class="alert alert-success">
    
**Task 4:** Implement a function `solve_Lotka_Volterra_explicit_Euler` that solves the Lotka-Volterra system by using the `explicit Euler method`. The inputs of the function are the initial values $x_0$, $y_0$ and time-step $\Delta t$. The function should return the solution history in time as the output.

Note that, an ODE system $d X/dt = f(X,t),\; \mbox{for} \;  X=(x,y)^\top$ can be solved by the explicit Euler iterations:
	\begin{equation*}
	X^{(i+1)}=X^{(i)} + \Delta t \cdot f(X^{(i)},t), \quad i=0,1,2,\dots, ({T}/{\Delta t}-1)
	\end{equation*}
	where $X^{(i)}$ denotes the approximated state vector $X$ at the discrete time step $t = \Delta t \cdot i$. For the example, we take $t_0=0.0$, $T=140.0$, $x(0)=10.0$, $y(0)=10.0$, $\alpha=0.1$, $\beta=0.02$, $\gamma=0.4$, $\delta=0.02$ and $\Delta t=0.001$. 
</div>

In [ ]:
def solve_Lotka_Volterra_explicit_Euler(x_0,y_0,deltat = 0.001):
    t_0 = 0.0
    T = 140.0
    
    x0 = [x_0]
    y0 = [y_0]
    t0 = [t_0]
    hist = [(x0,y0,t0)]
    
    alpha = 0.1
    beta = 0.02
    gamma = 0.4
    delta = 0.02
    
    time_steps = int(T/deltat)
    
    for i in range(time_steps):
        x = hist[0][0][i]
        y = hist[0][1][i]
        
        # use the explicit Euler method
        x = x + deltat * (alpha * x - beta * x * y)
        y = y + deltat * (-gamma * y + delta * x * y)
        t = deltat * (i+1)
        
        # add it to the solution history
        hist[0][0].append(x)
        hist[0][1].append(y)
        hist[0][2].append(t)
        
    return np.array(hist)

<div class="alert alert-success">
    
**Task 5:** Plot the solution history for the prey and predator population in time ($x$ vs. $t$ and $y$ vs. $t$ )
</div>

In [ ]:
slv = solve_Lotka_Volterra_explicit_Euler(10, 10, 0.001)
x_his = slv[0][0]
y_his = slv[0][1]
t_his = slv[0][2]

plt.figure()

plt.plot(t_his, x_his, label='prey (x vs t)')
plt.plot(t_his, y_his, label='predators (y vs t)')

plt.xlabel('time')
plt.ylabel('population')

plt.legend()

We now try to solve the same ODE system using a slightly different method. 

<div class="alert alert-success">
    
**Task 6:** Implement a function `solve_Lotka_Volterra_explicit_Heun` that solves the Lotka-Volterra system by using the `Heun's method`. The inputs of the function are the initial values $x_0,y_0$ and time-step $\Delta t$. The function should return the solution history in time as the output.

</div>

In [ ]:
def solve_Lotka_Volterra_explicit_Heun(x_0,y_0,deltat = 0.001):
    t_0 = 0.0
    T = 140.0
    
    x0 = [x_0]
    y0 = [y_0]
    t0 = [t_0]
    hist = [(x0,y0,t0)]
       
    time_steps = int(T/deltat)
    
    
    # using Heun's method
    for i in range(time_steps):
        x = hist[0][0][i]
        y = hist[0][1][i]
        fx = (0.1 * x) - (0.02 * x * y)
        fy = (-0.4 * y) + (0.02 * x * y)
        
        #predictor step
        x_predict = x + deltat * fx 
        y_predict = y + deltat * fy
        
        fx2 = (0.1 * x_predict) - (0.02 * x_predict * (y_predict+deltat))
        fy2 = (-0.4 * y_predict) + (0.02 * (x_predict+deltat) * y_predict)
        
        #corrector step
        x_correct = x + deltat * (fx + fx2)/2
        y_correct = y + deltat * (fy + fy2)/2
        
        t = deltat * (i+1)
        
        hist[0][0].append(x_correct)
        hist[0][1].append(y_correct)
        hist[0][2].append(t)
    
    return np.array(hist)

<div class="alert alert-success">
    
**Task 7:** Plot the solution history for the prey and predator population in time ($x$ vs. $t$ and $y$ vs. $t$). 
Compare your results with the results that are obtained by the explicit Euler method.
</div>

In [ ]:
# plotting the solution using heun's method
slvh = solve_Lotka_Volterra_explicit_Heun(10, 10, 0.001)
x_hish = slvh[0][0]
y_hish = slvh[0][1]
t_hish = slvh[0][2]

plt.figure()

plt.plot(t_hish, x_hish, label='heun: prey')
plt.plot(t_hish, y_hish, label='heun: predators')

plt.title("Heun's method")
plt.xlabel('time')
plt.ylabel('population')
plt.legend()


# plotting euler method
slve = solve_Lotka_Volterra_explicit_Euler(10, 10, 0.001)
x_hise = slve[0][0]
y_hise = slve[0][1]
t_hise = slve[0][2]

plt.figure()
plt.plot(t_hise, x_hise, label='euler: prey')
plt.plot(t_hise, y_hise, label='euler: predators')

plt.title('Euler method')
plt.xlabel('time')
plt.ylabel('population')
plt.legend()


# plotting heun and euler in the same graph
plt.figure()
plt.plot(t_hish, x_hish, label='heun: prey')
plt.plot(t_hish, y_hish, label='heun: predators')
plt.plot(t_hise, x_hise, label='euler: prey')
plt.plot(t_hise, y_hise, label='euler: predators')

plt.title('both methods')
plt.xlabel('time')
plt.ylabel('population')
plt.legend()


# plotting the difference of both methods
plt.figure()
plt.plot(t_hish, t_hish-t_hise, label='difference t')
plt.plot(t_hish, x_hish-x_hise, label='difference x')
plt.plot(t_hish, y_hish-y_hise, label='difference y')

plt.title('difference')
plt.xlabel('time')
plt.ylabel('difference')
plt.legend()


# Looking at the plots only, they look exactly the same. However, differences 
# show when one plots the differences of the different variables. While t 
# obviously has no differences at all (the used delta_t is the same after all), 
# x and y show fluctuations (sometimes Euler is bigger, sometimes Heun), which 
# get stronger with time